In [ ]:
import time
import sys
import threading
from web3 import Web3, exceptions
from tkinter import *
import webbrowser

INFURA_URL = "https://weathered-hardworking-glade.ethereum-goerli.discover.quiknode.pro/bf19c675a3260c97ed6d8691171a45a4e49bc2a1/"
UNISWAP_FACTORY_ADDRESS = "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f"
PAIR_CREATED_EVENT_SIGNATURE = "0x0d3648bd0f6ba80134a33ba9275ac585d9d315f0ad8355cddefde31afa28d0e9"
DEXTOOLS_BASE_URL = "https://www.dextools.io/app/es/ether/pair-explorer/"
ETHERSCAN_BASE_URL = "https://goerli.etherscan.io/tx/"
BLOCK_TIME = 15
SECONDS_IN_4_HOURS = 24 * 60 * 60
BLOCKS_IN_4_HOURS = SECONDS_IN_4_HOURS // BLOCK_TIME

w3 = Web3(Web3.HTTPProvider(INFURA_URL))

def open_link(url):
    webbrowser.open(url)

def process_event(event):
    event_signature = event['topics'][0].hex()
    tx_hash = event['transactionHash'].hex()

    if event_signature == PAIR_CREATED_EVENT_SIGNATURE:
        token0, token1 = event['topics'][1].hex()[24:], event['topics'][2].hex()[24:]

        etherscan_link = f"{ETHERSCAN_BASE_URL}{tx_hash}"
        dextools_link = f"{DEXTOOLS_BASE_URL}{token1}"

        
        output.insert(END, f"Block Number: {event['blockNumber']}\n")
        output.insert(END, f"Transaction Hash: {tx_hash}\n")
        output.insert(END, f"Token Pair: {token0} and {token1}\n")
        output.insert(END, "\n\n")

        output.insert(END, etherscan_link, ("link",))
        output.insert(END, "\n")
        output.insert(END, "\n\n")

        output.insert(END, dextools_link, ("link",))
        output.insert(END, "\n\n")
        output.yview(END)

def main():
    latest_block = w3.eth.block_number
    starting_block = max(latest_block - BLOCKS_IN_4_HOURS, 0)

    while True:
        try:
            new_block = w3.eth.block_number

            if new_block > latest_block:
                output.insert(END, f"New block detected: {new_block}\n")
                for event in w3.eth.get_logs({'fromBlock': starting_block, 'toBlock': new_block, 'address': UNISWAP_FACTORY_ADDRESS}):
                    process_event(event)

                latest_block = new_block
                starting_block = latest_block

            time.sleep(5)

        except exceptions.ConnectionError:
            output.insert(END, "Connection error, retrying...\n")
            time.sleep(10)

script_thread = threading.Thread(target=main)
script_thread.start()

root = Tk()
root.title("Uniswap Event Listener")
root.geometry("900x600")

output = Text(root, wrap='word', width=110, height=30, bg="#f0f0f0")
output.pack(padx=10, pady=10)

output.tag_configure("link", foreground="blue", underline=True)
output.tag_bind("link", "<Button-1>", lambda event: open_link(event.widget.get('insert linestart', 'insert lineend')))

output.config(state=NORMAL)

root.mainloop()


In [ ]:
!pyinstaller --onefile --noconsole new_lps.py


In [1]:
import json
from web3 import Web3
from web3.datastructures import AttributeDict
from hexbytes import HexBytes

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, HexBytes):
            return obj.hex()
        return super(CustomJSONEncoder, self).default(obj)

def recursively_convert_to_dict(obj):
    if isinstance(obj, list):
        return [recursively_convert_to_dict(item) for item in obj]
    elif isinstance(obj, dict) or isinstance(obj, AttributeDict):
        return {k: recursively_convert_to_dict(v) for k, v in obj.items()}
    else:
        return obj

# Connect to the Ethereum node using your RPC URL
w3 = Web3(Web3.HTTPProvider("https://weathered-hardworking-glade.ethereum-goerli.discover.quiknode.pro/bf19c675a3260c97ed6d8691171a45a4e49bc2a1/"))

# Set the number of blocks to go back
num_blocks_to_go_back = 100000

# Get the latest block number
latest_block_number = w3.eth.block_number

# Calculate the starting block number
starting_block_number = max(0, latest_block_number - num_blocks_to_go_back)

# Download the specified number of blocks
blocks = []
for block_number in range(starting_block_number, latest_block_number + 1):
    block = w3.eth.get_block(block_number, full_transactions=True)
    blocks.append(recursively_convert_to_dict(block))
    print(f"Block {block_number}")

# Save the blocks to a JSON file
with open('blocks.json', 'w') as f:
    json.dump(blocks, f, cls=CustomJSONEncoder)

print(f"Downloaded {len(blocks)} blocks from {starting_block_number} to {latest_block_number} and saved to blocks.json.")


Block 8830456
Block 8830457
Block 8830458
Block 8830459
Block 8830460
Block 8830461
Block 8830462
Block 8830463
Block 8830464
Block 8830465
Block 8830466
Block 8830467
Block 8830468
Block 8830469
Block 8830470
Block 8830471
Block 8830472
Block 8830473
Block 8830474
Block 8830475
Block 8830476
Block 8830477
Block 8830478
Block 8830479
Block 8830480
Block 8830481
Block 8830482
Block 8830483
Block 8830484
Block 8830485
Block 8830486
Block 8830487
Block 8830488
Block 8830489
Block 8830490
Block 8830491
Block 8830492
Block 8830493
Block 8830494
Block 8830495
Block 8830496
Block 8830497
Block 8830498
Block 8830499
Block 8830500
Block 8830501
Block 8830502
Block 8830503
Block 8830504
Block 8830505
Block 8830506
Block 8830507
Block 8830508
Block 8830509
Block 8830510
Block 8830511
Block 8830512
Block 8830513
Block 8830514
Block 8830515
Block 8830516
Block 8830517
Block 8830518
Block 8830519
Block 8830520
Block 8830521
Block 8830522
Block 8830523
Block 8830524
Block 8830525
Block 8830526
Block 

In [ ]:
import csv
from web3 import Web3
import datetime
import concurrent.futures

INFURA_URL = "https://radial-wild-wish.discover.quiknode.pro/11ffd288312f100b12adc0c2157ff96549dcee38/"

w3 = Web3(Web3.HTTPProvider(INFURA_URL))

# Get the current block number
current_block = w3.eth.block_number


# Calculate the block number for 2 years ago
two_years_ago = current_block - ( 1 * 24 * 60 * 60) // 15

# Create a list to store the block information
blocks = []

def get_block_info(block_number):
    block = w3.eth.get_block(block_number)

    # Extract the relevant information from the block
    timestamp = block['timestamp']
    date = datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    difficulty = block['difficulty']
    gas_limit = block['gasLimit']
    gas_used = block['gasUsed']
    transactions = block['transactions']
    #print("block_number:", block_number, "date:", date, "difficulty:", difficulty, "gas_limit:", gas_limit, "gas_used:", gas_used, "transactions:", transactions)

    return [block_number, date, difficulty, gas_limit, gas_used, transactions]

# Use the ThreadPoolExecutor to run the get_block_info function asynchronously
with concurrent.futures.ThreadPoolExecutor() as executor:
    block_numbers = range(two_years_ago, current_block + 1)
    results = [executor.submit(get_block_info, block_number) for block_number in block_numbers]

    for future in concurrent.futures.as_completed(results):
        block_info = future.result()
        blocks.append(block_info)

# Write the blocks to a CSV file
with open('blocks.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Block Number', 'Date', 'Difficulty', 'Gas Limit', 'Gas Used', 'Transactions'])
    writer.writerows

